In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier as GBDT
from sklearn.ensemble import ExtraTreesClassifier as ET
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.ensemble import AdaBoostClassifier as ADA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import numpy as np
x,y = make_classification(n_samples=6000)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5)

In [2]:
### 第一层模型
clfs = [ GBDT(n_estimators=100),
       RF(n_estimators=100),
       ET(n_estimators=100),
       ADA(n_estimators=100)
]
X_train_stack  = np.zeros((X_train.shape[0], len(clfs)))
X_test_stack = np.zeros((X_test.shape[0], len(clfs))) 

In [3]:
### 6折stacking
n_folds = 6
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=1)
for i,clf in enumerate(clfs):
#     print("分类器：{}".format(clf))
    X_stack_test_n = np.zeros((X_test.shape[0], n_folds))
    for j,(train_index,test_index) in enumerate(skf.split(X_train,y_train)):
                tr_x = X_train[train_index]
                tr_y = y_train[train_index]
                clf.fit(tr_x, tr_y)
                #生成stacking训练数据集
                X_train_stack [test_index, i] = clf.predict_proba(X_train[test_index])[:,1]
                X_stack_test_n[:,j] = clf.predict_proba(X_test)[:,1]
    #生成stacking测试数据集
    X_test_stack[:,i] = X_stack_test_n.mean(axis=1) 

In [4]:
###第二层模型LR
clf_second = LogisticRegression(solver="lbfgs")
clf_second.fit(X_train_stack,y_train)
pred = clf_second.predict_proba(X_test_stack)[:,1]
roc_auc_score(y_test,pred)#0.9946

0.9622561817095379

In [12]:
type(X_train)

numpy.ndarray

In [14]:
y_train

array([1, 1, 0, ..., 0, 0, 0])

In [15]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
# train_x, test_x, train_y, test_y = train_test_split(iris.data, iris.target, test_size=0.2)


In [17]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])